# INF552 Homework5:

Student Name : Hsin-Yu, Chang (1160173733)

## 1. Multi-class and Multi-Label Classification Using Support Vector Machine

### (a) Downloads the Anuran Calls (MFCCs) Data Set

In [28]:
import urllib.request
import zipfile
import pandas
import numpy as np
from sklearn.metrics import f1_score, precision_score

In [29]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00406/Anuran%20Calls%20(MFCCs).zip'
zipname = "Anuran Calls (MFCCs).zip"
#download a zip file from Web(URL)
urllib.request.urlretrieve(url, zipname)
mfcc = zipfile.ZipFile(zipname).open("Frogs_MFCCs.csv")
mfcc = pandas.read_csv(mfcc)

In [30]:
mfccX = mfcc.iloc[:,:-4]

In [31]:
mfccf = mfcc.Family
mfccg = mfcc.Genus
mfccs = mfcc.Species

In [390]:
mfccf.value_counts()

Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: Family, dtype: int64

In [391]:
mfccg.value_counts()

Adenomera        4150
Hypsiboas        1593
Ameerega          542
Dendropsophus     310
Leptodactylus     270
Scinax            148
Osteocephalus     114
Rhinella           68
Name: Genus, dtype: int64

In [392]:
mfccs.value_counts()

AdenomeraHylaedactylus    3478
HypsiboasCordobae         1121
AdenomeraAndre             672
Ameeregatrivittata         542
HypsiboasCinerascens       472
HylaMinuta                 310
LeptodactylusFuscus        270
ScinaxRuber                148
OsteocephalusOophagus      114
Rhinellagranulosa           68
Name: Species, dtype: int64

##### Factorize the Labels 

In [32]:
mfcc.Family, categf = pandas.factorize(mfccf)
mfcc.Genus, categg = pandas.factorize(mfccg)
mfcc.Species, categs = pandas.factorize(mfccs)

##### Choose 70% of the data randomly as the training set

In [33]:
import random
import math
traini = random.sample(range(mfcc.shape[0]), math.ceil(mfcc.shape[0]*0.7))
traini.sort()
testi = [i for i in range(mfcc.shape[0]) if i not in traini ]

In [34]:
ttrain = mfcc.iloc[traini,:].reset_index(drop=True)
test = mfcc.iloc[testi,:].reset_index(drop=True)

In [35]:
colX = [col for col in ttrain.columns.tolist() if 'MFCCs_' in col]

In [36]:
ttrainX = ttrain[colX]
ttrainf = ttrain.Family
ttraing = ttrain.Genus
ttrains = ttrain.Species

In [37]:
testX = test[colX]
testf = test.Family
testg = test.Genus
tests = test.Species

In [38]:
import warnings
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [39]:
import sys
sys.path
sys.path.append(r'/anaconda3/pkgs/libsvm-3.23/python')
sys.path.append(r'/anaconda3/pkgs/libsvm-3.23/tools')

In [40]:
import svm
from sklearn.datasets import dump_svmlight_file
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import confusion_matrix
from grid import *

In [41]:
def svmlightfile(X, label):
    dump_svmlight_file(X, label, 'mfcctrainL.dat', zero_based=True, multilabel=False)
    mfcctrainL = load_svmlight_file('mfcctrainL.dat')
    return mfcctrainL   

In [110]:
mfcctrainF = svmlightfile(ttrainX, ttrainf)
mfcctrainG = svmlightfile(ttrainX, ttraing)
mfcctrainS = svmlightfile(ttrainX, ttrains)

In [42]:
mfcctestF = svmlightfile(testX, testf)
mfcctestG = svmlightfile(testX, testg)
mfcctestS = svmlightfile(testX, tests)

### (b) Solving a multi-class and multi-label problem

#### i. Exact Match Ratio and Hamming Loss for evaluating Multi-Label Classification

- Hamming Loss: the fraction of the wrong labels to the total number of labels
- Exact Match Ratio: percentage of datapoints that have all their labels classified 100% correctly

In [106]:
def multilabelleva(trueLabels, predLabels): #List of np.arrays of each label  
    trueLabels = np.stack(trueLabels).T
    predLabels = np.stack(predLabels).T
    
    #hamming loss
    hammingloss = (sum(trueLabels[:,0]!=predLabels[:,0])+sum(trueLabels[:,1]!=predLabels[:,1])+sum(trueLabels[:,2]!=predLabels[:,2]))/(trueLabels.shape[0]*trueLabels.shape[1])
    print('Hamming Loss: %0.3f'% hammingloss)
    
    #exact match score
    trueLstr = np.array([str(int(trueLabels[i,0]))+str(int(trueLabels[i,1]))+str(int(trueLabels[i,2])) for i in range(trueLabels.shape[0])])
    predLstr = np.array([str(int(predLabels[i,0]))+str(int(predLabels[i,1]))+str(int(predLabels[i,2])) for i in range(predLabels.shape[0])])
    emr = sum(trueLstr== predLstr)/len(trueLstr)
    print('Exact Match Ratio: %0.2f'% emr)
    
    return hammingloss, emr

#### ii. SVM for each of the labels with using Gaussian kernels and one versus all classifiers

In [313]:
gpredLabelstr = [] # collecting label prediction from each label's classifier
gpredLabelste = []

##### Use LIBSVM find the best C and gamma using 10-fold Cross Validation

In [307]:
def paramSearch(X, label):
    dump_svmlight_file(X, label, 'mfcctrainL.dat', zero_based=True, multilabel=False)
    mfcctrainL = load_svmlight_file('mfcctrainL.dat')
    rate, param = find_parameters('mfcctrainL.dat', '-log2c -2,8,2 -log2g -5,3,2 -v 10')
    return mfcctrainL, rate, param      

##### Label- Family 

__>__ Parameter-searching for C and gamma

In [308]:
mfcctrainF, ratef, paramf = paramSearch(ttrainX, ttrainf)

gnuplot executable not found


[local] 4.0 -1.0 98.9081 (best c=16.0, g=0.5, rate=98.9081)
[local] 0.0 -1.0 97.0419 (best c=16.0, g=0.5, rate=98.9081)
[local] 4.0 -3.0 97.7566 (best c=16.0, g=0.5, rate=98.9081)
[local] 0.0 -3.0 94.9176 (best c=16.0, g=0.5, rate=98.9081)
[local] 8.0 -1.0 99.1066 (best c=256.0, g=0.5, rate=99.1066)
[local] 8.0 -3.0 98.5507 (best c=256.0, g=0.5, rate=99.1066)
[local] 4.0 3.0 98.9081 (best c=256.0, g=0.5, rate=99.1066)
[local] 0.0 3.0 98.8485 (best c=256.0, g=0.5, rate=99.1066)
[local] 8.0 3.0 98.9081 (best c=256.0, g=0.5, rate=99.1066)
[local] -2.0 -1.0 96.0889 (best c=256.0, g=0.5, rate=99.1066)
[local] -2.0 -3.0 93.6272 (best c=256.0, g=0.5, rate=99.1066)
[local] -2.0 3.0 98.0941 (best c=256.0, g=0.5, rate=99.1066)
[local] 4.0 -5.0 95.1757 (best c=256.0, g=0.5, rate=99.1066)
[local] 0.0 -5.0 93.1705 (best c=256.0, g=0.5, rate=99.1066)
[local] 8.0 -5.0 97.7566 (best c=256.0, g=0.5, rate=99.1066)
[local] -2.0 -5.0 83.7006 (best c=256.0, g=0.5, rate=99.1066)
[local] 6.0 -1.0 99.1463 (be

In [309]:
OvRsvc = OneVsRestClassifier(SVC(kernel='rbf', C=paramf['c'], gamma = paramf['g'], probability=True))
optimalsvcF = OvRsvc.fit(ttrainX, mfcctrainF[1])

In [314]:
sksvctrF = optimalsvcF.predict(ttrainX)
gpredLabelstr.append(sksvctrF)
sksvcteF = optimalsvcF.predict(testX)
gpredLabelste.append(sksvcteF)

In [321]:
print("Evaluation of 'Family' label on Training Set:") 
print(confusion_matrix(mfcctrainF[1], sksvctrF))
print('F1-score: %0.3f' % f1_score(mfcctrainF[1], sksvctrF, average='weighted')) 

Evaluation of 'Family' label on Training Set:
[[3092    0    0    0]
 [   0  377    0    0]
 [   0    0 1522    0]
 [   0    0    0   46]]
F1-score: 1.000


In [322]:
print("Evaluation of 'Family' label on Test Set:") 
print(confusion_matrix(mfcctestF[1], sksvcteF))
print('F1-score: %0.3f' % f1_score(mfcctestF[1], sksvcteF, average='weighted')) 

Evaluation of 'Family' label on Test Set:
[[1321    1    6    0]
 [   0  164    1    0]
 [   4    0  639    0]
 [   2    0    3   17]]
F1-score: 0.911


##### Label- Genus

__>__ Parameter-searching for C and gamma

In [324]:
mfcctrainG, rateg, paramg = paramSearch(ttrainX, ttraing)

gnuplot executable not found


[local] 4.0 -1.0 98.9081 (best c=16.0, g=0.5, rate=98.9081)
[local] 0.0 -1.0 97.3397 (best c=16.0, g=0.5, rate=98.9081)
[local] 4.0 -3.0 97.8559 (best c=16.0, g=0.5, rate=98.9081)
[local] 0.0 -3.0 93.8654 (best c=16.0, g=0.5, rate=98.9081)
[local] 8.0 -1.0 98.8287 (best c=16.0, g=0.5, rate=98.9081)
[local] 8.0 -3.0 98.65 (best c=16.0, g=0.5, rate=98.9081)
[local] 4.0 3.0 98.3323 (best c=16.0, g=0.5, rate=98.9081)
[local] 0.0 3.0 98.1338 (best c=16.0, g=0.5, rate=98.9081)
[local] 8.0 3.0 98.3323 (best c=16.0, g=0.5, rate=98.9081)
[local] -2.0 -1.0 94.1036 (best c=16.0, g=0.5, rate=98.9081)
[local] -2.0 -3.0 89.7757 (best c=16.0, g=0.5, rate=98.9081)
[local] -2.0 3.0 96.7044 (best c=16.0, g=0.5, rate=98.9081)
[local] 4.0 -5.0 95.7911 (best c=16.0, g=0.5, rate=98.9081)
[local] 0.0 -5.0 89.6962 (best c=16.0, g=0.5, rate=98.9081)
[local] 8.0 -5.0 97.7367 (best c=16.0, g=0.5, rate=98.9081)
[local] -2.0 -5.0 79.194 (best c=16.0, g=0.5, rate=98.9081)
[local] 6.0 -1.0 98.8684 (best c=16.0, g=0.

In [325]:
OvRsvc = OneVsRestClassifier(SVC(kernel='rbf', C=paramg['c'], gamma = paramg['g'], probability=True))
#optimalsvc = GridSearchCV(OvRsvc, param_grid=parameters, cv=10, scoring='neg_log_loss')
optimalsvcG = OvRsvc.fit(ttrainX, mfcctrainG[1])

In [326]:
sksvctrG = optimalsvcG.predict(ttrainX)
gpredLabelstr.append(sksvctrG)
sksvcteG = optimalsvcG.predict(testX)
gpredLabelste.append(sksvcteG)

In [327]:
print("Evaluation of 'Genus' label on Training Set:") 
print(confusion_matrix(mfcctrainG[1], sksvctrG))
print('F1-score: %0.3f' % f1_score(mfcctrainG[1], sksvctrG, average='weighted')) 

Evaluation of 'Genus' label on Training Set:
[[2915    0    0    0    0    0    0    0]
 [   0  377    0    0    0    0    0    0]
 [   0    0  221    0    0    0    0    0]
 [   0    0    0 1116    0    0    0    0]
 [   0    0    0    0  177    0    0    0]
 [   0    0    0    0    0   80    0    0]
 [   0    0    0    0    0    0   46    0]
 [   0    0    0    0    0    0    0  105]]
F1-score: 1.000


In [328]:
print("Evaluation of 'Genus' label on Test Set:") 
print(confusion_matrix(mfcctestG[1], sksvcteG))
print('F1-score: %0.3f' % f1_score(mfcctestG[1], sksvcteG, average='weighted')) 

Evaluation of 'Genus' label on Test Set:
[[1229    1    2    1    0    1    0    1]
 [   0  165    0    0    0    0    0    0]
 [   5    0   84    0    0    0    0    0]
 [   1    0    0  473    0    2    0    1]
 [   0    0    0    1   92    0    0    0]
 [   1    0    0    1    0   32    0    0]
 [   2    0    0    1    1    0   18    0]
 [   0    0    0    0    0    0    0   43]]
F1-score: 0.990


##### Label- Species

__>__ Parameter-searching for C and gamma

In [329]:
mfcctrainS, rates, params = paramSearch(ttrainX, ttrains)

gnuplot executable not found


[local] 4.0 -1.0 98.8684 (best c=16.0, g=0.5, rate=98.8684)
[local] 0.0 -1.0 97.5978 (best c=16.0, g=0.5, rate=98.8684)
[local] 4.0 -3.0 98.1934 (best c=16.0, g=0.5, rate=98.8684)
[local] 0.0 -3.0 94.6595 (best c=16.0, g=0.5, rate=98.8684)
[local] 8.0 -1.0 98.8088 (best c=16.0, g=0.5, rate=98.8684)
[local] 8.0 -3.0 98.9279 (best c=256.0, g=0.125, rate=98.9279)
[local] 4.0 3.0 97.9353 (best c=256.0, g=0.125, rate=98.9279)
[local] 0.0 3.0 97.7765 (best c=256.0, g=0.125, rate=98.9279)
[local] 8.0 3.0 97.9353 (best c=256.0, g=0.125, rate=98.9279)
[local] -2.0 -1.0 94.7588 (best c=256.0, g=0.125, rate=98.9279)
[local] -2.0 -3.0 90.6095 (best c=256.0, g=0.125, rate=98.9279)
[local] -2.0 3.0 96.4264 (best c=256.0, g=0.125, rate=98.9279)
[local] 4.0 -5.0 96.9823 (best c=256.0, g=0.125, rate=98.9279)
[local] 0.0 -5.0 90.669 (best c=256.0, g=0.125, rate=98.9279)
[local] 8.0 -5.0 98.372 (best c=256.0, g=0.125, rate=98.9279)
[local] -2.0 -5.0 83.0256 (best c=256.0, g=0.125, rate=98.9279)
[local] 6

In [331]:
OvRsvc = OneVsRestClassifier(SVC(kernel='rbf', C=params['c'], gamma = params['g'], probability=True))
#optimalsvc = GridSearchCV(OvRsvc, param_grid=parameters, cv=10, scoring='neg_log_loss')
optimalsvcS = OvRsvc.fit(ttrainX, mfcctrainS[1])

In [332]:
sksvctrS = optimalsvcS.predict(ttrainX)
gpredLabelstr.append(sksvctrS)
sksvcteS = optimalsvcS.predict(testX)
gpredLabelste.append(sksvcteS)

In [333]:
print("Evaluation of 'Species' label on Training Set:") 
print(confusion_matrix(mfcctrainS[1], sksvctrS))
print('F1-score: %0.3f' % f1_score(mfcctrainS[1], sksvctrS, average='weighted')) 

Evaluation of 'Species' label on Training Set:
[[ 462    0    0    1    0    0    0    0    0    0]
 [   0  377    0    0    0    0    0    0    0    0]
 [   0    0 2452    0    0    0    0    0    0    0]
 [   1    0    0  220    0    0    0    0    0    0]
 [   0    0    0    0  331    1    0    0    0    0]
 [   0    0    0    0    0  784    0    0    0    0]
 [   0    0    0    0    0    0  177    0    0    0]
 [   0    0    0    0    2    0    0   78    0    0]
 [   0    0    0    0    0    0    0    0   46    0]
 [   0    0    0    0    0    0    0    0    0  105]]
F1-score: 0.999


In [334]:
print("Evaluation of 'Species' label on Test Set:") 
print(confusion_matrix(mfcctestS[1], sksvcteS))
print('F1-score: %0.3f' % f1_score(mfcctestS[1], sksvcteS, average='weighted')) 

Evaluation of 'Species' label on Test Set:
[[ 206    1    0    1    0    0    0    1    0    0]
 [   0  164    0    1    0    0    0    0    0    0]
 [   0    0 1024    1    0    0    0    0    0    1]
 [   1    0    3   85    0    0    0    0    0    0]
 [   1    0    0    0  136    0    0    3    0    0]
 [   0    0    0    0    0  334    0    2    0    1]
 [   0    0    0    0    1    0   92    0    0    0]
 [   1    0    0    0    0    0    0   33    0    0]
 [   3    0    1    0    0    0    1    0   17    0]
 [   0    0    0    0    0    0    0    0    0   43]]
F1-score: 0.989


##### Model Evaluation

In [111]:
trueLabelstr = [mfcctrainF[1], mfcctrainG[1], mfcctrainS[1]]
trueLabelste = [mfcctestF[1], mfcctestG[1], mfcctestS[1]]

In [345]:
print('SVM with OvR Gaussian kernels on Training Set')
ghltr, gemrtr = multilabelleva(trueLabelstr, gpredLabelstr)

SVM with OvR Gaussian kernels on Training Set
Hamming Loss: 0.000
Exact Match Ratio: 1.00


In [346]:
print('SVM with OvR Gaussian kernels on Test Set')
ghlte, gemrte = multilabelleva(trueLabelste, gpredLabelste)

SVM with OvR Gaussian kernels on Test Set
Hamming Loss: 0.010
Exact Match Ratio: 0.99


##### Model Evaluation of ii. 

__>__ SVM with OvR Gaussian kernels has a good performance both on training and test set, even though there are class-inbalanced in each label, but this multilabel-multiclassifier have a high exact match ratio (closs to 100%) and a low lamming loss.

#### iii. L1-penalized SVMs

In [372]:
l1predLabelstr = []
l1predLabelste = []

In [59]:
#import sys
#sys.path
sys.path.append(r'/anaconda3/pkgs/liblinear-2.21/python')

In [60]:
import liblinear
from liblinearutil import *

__>__ Parameter-searching for C in L1-penalized SVM

In [55]:
def L1penalySVM(mfcctrainL):
    acc = []
    C = [0.01, 0.1, 1, 10, 100, 1000, 10000]
    parm = '-s 5 -v 10 -c '
    for c in C:
        acc.append(train(mfcctrainL[1], mfcctrainL[0], parm+str(c)))
    bestC = np.array(C)[np.array(acc)==np.array(acc).max()][0]
    model = train(mfcctrainL[1], mfcctrainL[0], '-s 5 -c '+str(bestC))
    return bestC, model 

In [56]:
import warnings
def predictL1(svmfile, model):
    predy, _, proby = predict(svmfile[1], svmfile[0], model)
    print(confusion_matrix(svmfile[1], predy))
    warnings.filterwarnings("ignore")
    print('F1-score: %0.3f' % f1_score(svmfile[1], predy, average='weighted'))
    return np.array(predy)    

##### Label- Family 

In [351]:
bestCf, modelL1F = L1penalySVM(mfcctrainF)
print("Optimal C of L1-penalized for 'Family' Label: %0.2f" % bestCf)

Cross Validation Accuracy = 87.8301%
Cross Validation Accuracy = 93.0713%
Cross Validation Accuracy = 93.4683%
Cross Validation Accuracy = 93.4286%
Cross Validation Accuracy = 93.3691%
Cross Validation Accuracy = 93.3492%
Cross Validation Accuracy = 93.4485%
Optimal C of L1-penalized for 'Family' Label: 1.00


In [373]:
print("Evaluation of 'Family' label on Training Set:") 
l1predLabelstr.append(predictL1(mfcctrainF, modelL1F))

Evaluation of 'Family' label on Training Set:
Accuracy = 93.6272% (4716/5037) (classification)
[[3002   24   66    0]
 [   8  340   29    0]
 [ 130   18 1374    0]
 [   3    0   43    0]]
F1-score: 0.932


In [468]:
print("Evaluation of 'Family' label on Test Set:") 
l1predLabelste.append(predictL1(mfcctestF, modelL1F))

Evaluation of 'Family' label on Test Set:
Accuracy = 93.6052% (2020/2158) (classification)
[[1286   16   26    0]
 [   6  147   12    0]
 [  45   11  587    0]
 [   4    0   18    0]]
F1-score: 0.931


__>__ L1-penalized SVM on 'Family' label has a problem on dealing with class-inbalance label, it missiclassifies most of 'Bufonidae' class to 'Hylidae' class, which is the second majority of 'Family' label.  

##### Label- Genus

In [368]:
bestCg, modelL1G = L1penalySVM(mfcctrainG)
print("Optimal C of L1-penalized for 'Genus' Label: %0.2f" % bestCg)

Cross Validation Accuracy = 83.0256%
Cross Validation Accuracy = 92.3963%
Cross Validation Accuracy = 93.9647%
Cross Validation Accuracy = 94.6794%
Cross Validation Accuracy = 94.8978%
Cross Validation Accuracy = 95.0169%
Cross Validation Accuracy = 94.9772%
Optimal C of L1-penalized for 'Genus' Label: 1000.00


In [375]:
print("Evaluation of 'Genus' label on Training Set:") 
l1predLabelstr.append(predictL1(mfcctrainG, modelL1G))

Evaluation of 'Genus' label on Training Set:
Accuracy = 95.3147% (4801/5037) (classification)
[[2885   15    3   11    0    0    0    1]
 [   7  363    5    2    0    0    0    0]
 [  56    7  148   10    0    0    0    0]
 [  27    0    0 1083    4    0    0    2]
 [   3    0    4   17  152    1    0    0]
 [   3    0    0   31    0   46    0    0]
 [   5    0    0   17    1    0   23    0]
 [   1    0    0    3    0    0    0  101]]
F1-score: 0.950


In [376]:
print("Evaluation of 'Genus' label on Test Set:") 
l1predLabelste.append(predictL1(mfcctestG, modelL1G))

Evaluation of 'Genus' label on Test Set:
Accuracy = 94.671% (2043/2158) (classification)
[[1214   13    0    5    0    1    0    2]
 [   6  156    3    0    0    0    0    0]
 [  28    4   55    2    0    0    0    0]
 [   5    1    0  468    2    1    0    0]
 [   3    0    0    7   83    0    0    0]
 [   2    0    0   18    0   14    0    0]
 [   5    0    0    4    0    0   13    0]
 [   0    0    0    3    0    0    0   40]]
F1-score: 0.943


##### Label- Species

In [371]:
bestCs, modelL1S = L1penalySVM(mfcctrainS)
print("Optimal C of L1-penalized for 'Species' Label: %0.2f" % bestCs)

Cross Validation Accuracy = 82.1322%
Cross Validation Accuracy = 92.7536%
Cross Validation Accuracy = 95.4139%
Cross Validation Accuracy = 95.7713%
Cross Validation Accuracy = 95.7713%
Cross Validation Accuracy = 95.672%
Cross Validation Accuracy = 95.5926%
Optimal C of L1-penalized for 'Species' Label: 10.00


In [377]:
print("Evaluation of 'Species' label on Training Set:") 
l1predLabelstr.append(predictL1(mfcctrainS, modelL1S))

Evaluation of 'Species' label on Training Set:
Accuracy = 96.1286% (4842/5037) (classification)
[[ 439   14    0    1    0    7    0    0    0    2]
 [   2  364    0    6    1    2    0    2    0    0]
 [   0    1 2449    0    1    1    0    0    0    0]
 [  23    8   19  159    1   11    0    0    0    0]
 [   7    0    0    1  311   12    1    0    0    0]
 [   0    0    5    1    4  767    5    0    0    2]
 [   2    0    0    4    3    8  160    0    0    0]
 [   2    0    0    0   11   16    1   49    1    0]
 [   1    0    0    0    0    2    3    0   40    0]
 [   0    0    0    0    0    1    0    0    0  104]]
F1-score: 0.960


In [378]:
print("Evaluation of 'Species' label on Test Set:") 
l1predLabelste.append(predictL1(mfcctestS, modelL1S))

Evaluation of 'Species' label on Test Set:
Accuracy = 95.8295% (2068/2158) (classification)
[[ 193   10    0    0    0    5    0    1    0    0]
 [   3  157    0    4    0    0    0    1    0    0]
 [   0    0 1025    0    0    0    0    0    0    1]
 [   8    4   11   64    0    2    0    0    0    0]
 [   2    0    0    0  135    3    0    0    0    0]
 [   0    0    1    0    3  329    3    0    0    1]
 [   2    0    0    0    1    5   85    0    0    0]
 [   0    0    0    0    3    9    0   21    1    0]
 [   4    0    0    0    0    1    0    0   17    0]
 [   0    0    0    1    0    0    0    0    0   42]]
F1-score: 0.957


##### Model Evaluation of iii.

In [379]:
print('OvR L1-penalized SVM on Training Set')
l1hltr, l1emrtr = multilabelleva(trueLabelstr, l1predLabelstr)

OvR L1-penalized SVM on Training Set
Hamming Loss: 0.050
Exact Match Ratio: 0.92


In [380]:
print('OvR L1-penalized SVM on Test Set')
l1hlte, l1emrte = multilabelleva(trueLabelste, l1predLabelste)

OvR L1-penalized SVM on Test Set
Hamming Loss: 0.053
Exact Match Ratio: 0.91


__>__ Each label classifier building on L1-penalized SVM has problem on misclassifying minority class, which makes this multi-label classifier have higher hamming loss and lower exact match ratio comparing to the RBF SVM classifier in question ii. 

#### iv. Remedy Class Imbalance on L1-penalized SVMs

__>__ To remedy class-imbalance in each label, I set class weights, with minority having higher cost weight when building classifer. 

In [446]:
wl1predLabelstr = []
wl1predLabelste = []

##### Label- Family

__>__ Balancing class weights in 'Family' label

In [381]:
mfccf.value_counts()

Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: Family, dtype: int64

In [395]:
categf

Index(['Leptodactylidae', 'Dendrobatidae', 'Hylidae', 'Bufonidae'], dtype='object')

In [434]:
from sklearn.utils import class_weight
weightf = class_weight.compute_class_weight('balanced', np.unique(mfcc.Family), mfcc.Family)
weightf

array([ 0.40695701,  3.31872694,  0.83083141, 26.45220588])

In [429]:
acc = []
C = [0.01, 0.1, 1, 10, 100, 1000, 10000]
parm = '-s 5 -w0 0.4 -w1 3 -w2 0.8 -w3 26 -v 10 -c '
for c in C:
    acc.append(train(mfcctrainF[1], mfcctrainF[0], parm+str(c)))
sbestCf = np.array(C)[np.array(acc)==np.array(acc).max()][0]
smodelF = train(mfcctrainF[1], mfcctrainF[0], '-s 5 -w0 0.4 -w1 3 -w2 0.8 -w3 26 -c '+str(sbestCf))
print("Optimal C of L1-penalized for 'Family' Label on weighting: %0.2f" % sbestCf)

Cross Validation Accuracy = 82.2513%
Cross Validation Accuracy = 91.6816%
Cross Validation Accuracy = 93.0713%
Cross Validation Accuracy = 93.111%
Cross Validation Accuracy = 93.1705%
Cross Validation Accuracy = 93.0911%
Cross Validation Accuracy = 93.2698%
Optimal C of L1-penalized for 'Family' Label on weighting: 10000.00


In [447]:
print("Evaluation of 'Family' label on Training Set:") 
wl1predLabelstr.append(predictL1(mfcctrainF, smodelF))

Evaluation of 'Family' label on Training Set:
Accuracy = 93.3889% (4704/5037) (classification)
[[2944   45   79   24]
 [   6  365    5    1]
 [ 100   41 1354   27]
 [   3    0    2   41]]
F1-score: 0.936


In [448]:
print("Evaluation of 'Family' label on Test Set:") 
wl1predLabelste.append(predictL1(mfcctestF, smodelF))

Evaluation of 'Family' label on Test Set:
Accuracy = 93.6052% (2020/2158) (classification)
[[1264   27   31    6]
 [   2  156    6    1]
 [  37   17  582    7]
 [   3    0    1   18]]
F1-score: 0.937


__>__ With weighting, the model on 'Family' label performs better classification in the minority class comparing to previous model, especially on the 'Bufonidae' class, which has mostly been misclassified to 'Hylidae' without weighting.  

##### Label- Genus

In [405]:
mfccg.value_counts()

Adenomera        4150
Hypsiboas        1593
Ameerega          542
Dendropsophus     310
Leptodactylus     270
Scinax            148
Osteocephalus     114
Rhinella           68
Name: Genus, dtype: int64

In [406]:
categg

Index(['Adenomera', 'Ameerega', 'Dendropsophus', 'Hypsiboas', 'Leptodactylus',
       'Osteocephalus', 'Rhinella', 'Scinax'],
      dtype='object')

In [435]:
weightg = class_weight.compute_class_weight('balanced', np.unique(mfcc.Genus), mfcc.Genus)
weightg

array([ 0.21671687,  1.65936347,  2.90120968,  0.56457941,  3.33101852,
        7.88925439, 13.22610294,  6.07685811])

In [436]:
acc = []
C = [0.01, 0.1, 1, 10, 100, 1000, 10000]
parm = '-s 5 -w0 0.2 -w1 1.6 -w2 3 -w3 0.5 -w4 3 -w5 8 -w6 13 -w7 6 -v 10 -c '
for c in C:
    acc.append(train(mfcctrainG[1], mfcctrainG[0], parm+str(c)))
sbestCg = np.array(C)[np.array(acc)==np.array(acc).max()][0]
smodelG = train(mfcctrainG[1], mfcctrainG[0], '-s 5 -w0 0.2 -w1 1.6 -w2 3 -w3 0.5 -w4 3 -w5 8 -w6 13 -w7 6 -c '+str(sbestCg))
print("Optimal C of L1-penalized for 'Genus' Label on weighting: %0.2f" % sbestCg)

Cross Validation Accuracy = 82.3705%
Cross Validation Accuracy = 93.786%
Cross Validation Accuracy = 94.4014%
Cross Validation Accuracy = 94.3816%
Cross Validation Accuracy = 94.3816%
Cross Validation Accuracy = 94.5801%
Cross Validation Accuracy = 94.4213%
Optimal C of L1-penalized for 'Genus' Label on weighting: 1000.00


In [449]:
print("Evaluation of 'Genus' label on Training Set:") 
wl1predLabelstr.append(predictL1(mfcctrainG, smodelG))

Evaluation of 'Genus' label on Training Set:
Accuracy = 94.9772% (4784/5037) (classification)
[[2815   38   30   13    0    9    8    2]
 [   4  354   17    1    0    0    1    0]
 [  27    6  176    6    0    0    5    1]
 [  16    0    2 1072   11    7    6    2]
 [   0    0    5   10  160    1    1    0]
 [   0    0    0   10    0   65    5    0]
 [   1    0    0    1    2    0   42    0]
 [   0    0    0    0    0    0    5  100]]
F1-score: 0.951


In [450]:
print("Evaluation of 'Genus' label on Test Set:") 
wl1predLabelste.append(predictL1(mfcctestG, smodelG))

Evaluation of 'Genus' label on Test Set:
Accuracy = 94.8563% (2047/2158) (classification)
[[1188   26    6    5    0    6    2    2]
 [   1  155    8    0    0    0    1    0]
 [  16    3   67    0    0    0    3    0]
 [   4    0    0  463    3    5    0    2]
 [   0    0    0    5   86    0    2    0]
 [   0    0    0    5    0   27    2    0]
 [   3    0    0    1    0    0   18    0]
 [   0    0    0    0    0    0    0   43]]
F1-score: 0.949


__>__ The 'Genus' model with a weighting remedy performs not only better classification on 'Osteocephalus' class comparing to previous model, which has been misclassified to 'Leptodactylus' class without weighting, but also a better classification on 'Dendropsophus' class.

##### Label- Species

In [440]:
mfccs.value_counts()

AdenomeraHylaedactylus    3478
HypsiboasCordobae         1121
AdenomeraAndre             672
Ameeregatrivittata         542
HypsiboasCinerascens       472
HylaMinuta                 310
LeptodactylusFuscus        270
ScinaxRuber                148
OsteocephalusOophagus      114
Rhinellagranulosa           68
Name: Species, dtype: int64

In [441]:
categs

Index(['AdenomeraAndre', 'Ameeregatrivittata', 'AdenomeraHylaedactylus',
       'HylaMinuta', 'HypsiboasCinerascens', 'HypsiboasCordobae',
       'LeptodactylusFuscus', 'OsteocephalusOophagus', 'Rhinellagranulosa',
       'ScinaxRuber'],
      dtype='object')

In [442]:
weights = class_weight.compute_class_weight('balanced', np.unique(mfcc.Species), mfcc.Species)
weights

array([ 1.07068452,  1.32749077,  0.20687177,  2.32096774,  1.52436441,
        0.64183764,  2.66481481,  6.31140351, 10.58088235,  4.86148649])

In [443]:
acc = []
C = [0.01, 0.1, 1, 10, 100, 1000, 10000]
parm = '-s 5 -w0 1 -w1 1 -w2 0.2 -w3 2.3 -w4 1.5 -w5 0.6 -w6 2.6 -w7 6 -w8 10 -w9 5 -v 10 -c '
for c in C:
    acc.append(train(mfcctrainS[1], mfcctrainS[0], parm+str(c)))
sbestCs = np.array(C)[np.array(acc)==np.array(acc).max()][0]
smodelS = train(mfcctrainS[1], mfcctrainS[0], '-s 5 -w0 1 -w1 1 -w2 0.2 -w3 2.3 -w4 1.5 -w5 0.6 -w6 2.6 -w7 6 -w8 10 -w9 5 -c '+str(sbestCg))
print("Optimal C of L1-penalized for 'Species' Label on weighting: %0.2f" % sbestCs)

Cross Validation Accuracy = 86.2617%
Cross Validation Accuracy = 95.0169%
Cross Validation Accuracy = 95.6125%
Cross Validation Accuracy = 95.8309%
Cross Validation Accuracy = 95.6323%
Cross Validation Accuracy = 95.811%
Cross Validation Accuracy = 95.6125%
Optimal C of L1-penalized for 'Species' Label on weighting: 10.00


In [451]:
print("Evaluation of 'Species' label on Training Set:") 
wl1predLabelstr.append(predictL1(mfcctrainS, smodelS))

Evaluation of 'Species' label on Training Set:
Accuracy = 96.3073% (4851/5037) (classification)
[[ 433   14    0    6    0    3    0    3    1    3]
 [   2  350    0   18    1    1    0    3    2    0]
 [   0    2 2447    0    1    1    0    0    1    0]
 [  16    6    9  179    1    4    0    0    5    1]
 [   5    0    0    1  312    6    2    4    2    0]
 [   1    0    5    1    6  760    7    0    2    2]
 [   1    0    0    4    2    4  164    1    1    0]
 [   1    0    0    0    6    6    1   62    4    0]
 [   0    0    0    0    0    0    2    0   44    0]
 [   0    0    0    0    0    0    0    0    5  100]]
F1-score: 0.963


In [452]:
print("Evaluation of 'Species' label on Test Set:") 
wl1predLabelste.append(predictL1(mfcctestS, smodelS))

Evaluation of 'Species' label on Test Set:
Accuracy = 95.5978% (2063/2158) (classification)
[[ 192   10    0    0    1    2    0    3    0    1]
 [   2  154    0    7    0    0    0    1    1    0]
 [   0    0 1020    3    1    0    0    0    0    2]
 [   7    3   10   67    0    1    0    0    1    0]
 [   2    0    0    0  134    1    1    2    0    0]
 [   0    0    1    0    4  323    3    3    1    2]
 [   0    0    0    0    1    3   87    0    2    0]
 [   0    0    0    0    2    5    1   25    1    0]
 [   3    0    0    0    0    1    0    0   18    0]
 [   0    0    0    0    0    0    0    0    0   43]]
F1-score: 0.956


__>__ In 'Species' class, class imbalance does not cause large problem in previous model with f1-score = 0.957 on test set, but in this exercise, I still use weighting remedy to fix the class imbalance in the 'Species' label, but both accuracy and f1-score do not seem better than previous model without weighting.

##### Model Evaluation of iv.

In [453]:
print('OvR L1-penalized SVM on Training Set with weighting')
wl1hltr, wl1emrtr = multilabelleva(trueLabelstr, wl1predLabelstr)

OvR L1-penalized SVM on Training Set with weighting
Hamming Loss: 0.051
Exact Match Ratio: 0.91


In [454]:
print('OvR L1-penalized SVM on Test Set with weighting')
wl1hlte, wl1emrte = multilabelleva(trueLabelste, wl1predLabelste)

OvR L1-penalized SVM on Test Set with weighting
Hamming Loss: 0.053
Exact Match Ratio: 0.91


#### __>__ Conclusion of Model Selection on the Anuran Calls (MFCCs) Data Set:

In this Multi-class and Multi-label data set, comparing this three model based on exact match Ratio and hamming loss, SVM with Gaussian kernel and OvR classifiers has an outstanding performance with exact match ratio = 0.99 and hamming loss = 0.01.  

## 2. K-Means Clustering on a Multi-class and Multi-Label Data Set

### (a) K-means Clustering with optimal k based on Silhouettes

In [9]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
def silhouettescore(n_clusters, X, rs):
    silhouettescore = []
    for clus in range(2, n_clusters+5):
        clusterer = KMeans(n_clusters=clus, random_state = rs)
        cluster_labels = clusterer.fit_predict(X)
        silhouettescore.append(silhouette_score(X, cluster_labels))
    silhouettescore = np.array(silhouettescore)
    
#     return [np.array([clus for clus in range(2, n_clusters+5)])[silhouettescore==max(silhouettescore)]
#             , max(silhouettescore)]
    return int(np.array([clus for clus in range(2, n_clusters+5)])[silhouettescore==max(silhouettescore)])

### (b) Label Majority in each cluster

In [23]:
def clustormajority(cluster_labels, label):
    clustermajority = []
    cluster = list(set(cluster_labels))
    cluster.sort()
    for c in cluster:
        #nparray index
        index, = np.where(cluster_labels == c)
        incluster = label[index]
        incluster = incluster.tolist()
        clustermajority.append(max(set(incluster), key = incluster.count))
    clustermajority = pandas.Series(clustermajority)
    return clustermajority    

In [24]:
def clusLabel(cluster_labels, clusmaj):
    clabel = np.empty(len(cluster_labels), dtype="U20")
    cluster = list(set(cluster_labels))
    for c in cluster:
        #nparray index
        index, = np.where(cluster_labels == c)
        clabel[index] = clusmaj[c]
    return clabel    

__>__ Demo of one simulation:

In [131]:
n_clusters = 10
demooptimalclus = silhouettescore(n_clusters, mfccX, 0) 
print('Best k based on Silhouettes: %d ' %demooptimalclus)
democlusterer = KMeans(n_clusters=demooptimalclus, random_state = 0)
democluster_labels = democlusterer.fit_predict(mfccX)
        
democlusmaj1 = clustormajority(democluster_labels, mfccf)
print('\n')  
print('Family Label for each cluster:')
print(democlusmaj1)

democlusmaj2 = clustormajority(democluster_labels, mfccg)
print('\n')  
print('Genus Label for each cluster:')
print(democlusmaj2)

democlusmaj3 = clustormajority(democluster_labels, mfccs)
print('\n')  
print('Species Label for each cluster:')
print(democlusmaj3)
        
democlusLabel1 = clusLabel(democluster_labels, democlusmaj1)
democlusLabel2 = clusLabel(democluster_labels, democlusmaj2)
democlusLabel3 = clusLabel(democluster_labels, democlusmaj3)
print('\n')        
print('hamming loss= %0.2f' % ((sum(mfccf != democlusLabel1)+sum(mfccg != democlusLabel2)+sum(mfccs != democlusLabel3))/(mfccX.shape[0]*3)))
  

Best k based on Silhouettes: 4 


Family Label for each cluster:
0    Leptodactylidae
1      Dendrobatidae
2            Hylidae
3            Hylidae
dtype: object


Genus Label for each cluster:
0    Adenomera
1     Ameerega
2    Hypsiboas
3    Hypsiboas
dtype: object


Species Label for each cluster:
0    AdenomeraHylaedactylus
1        Ameeregatrivittata
2      HypsiboasCinerascens
3         HypsiboasCordobae
dtype: object


hamming loss= 0.38


### __>__ Function of whole Monte-Carlo Simulation

In [25]:
def MonteCarlo(n, X, label1, label2, label3):
    hamming = []
    n_clusters = 10
    for i in range(n):
        optimalclus = silhouettescore(n_clusters, X, i) 
        #print(optimalclus)
        clusterer = KMeans(n_clusters=optimalclus, random_state = i)
        cluster_labels = clusterer.fit_predict(X)
        
        clusmaj1 = clustormajority(cluster_labels, label1)
        print(clusmaj1)
        clusmaj2 = clustormajority(cluster_labels, label2)
        print(clusmaj2)
        clusmaj3 = clustormajority(cluster_labels, label3)
        print(clusmaj3)
        
        clusLabel1 = clusLabel(cluster_labels, clusmaj1)
        clusLabel2 = clusLabel(cluster_labels, clusmaj2)
        clusLabel3 = clusLabel(cluster_labels, clusmaj3)
        
        hammingloss = (sum(label1 != clusLabel1)+sum(label2 != clusLabel2)+sum(label3 != clusLabel3))/(X.shape[0]*3)
        hamming.append(hammingloss) 
    return np.array(hamming)

In [464]:
MCsimulation = MonteCarlo(50, mfccX, mfccf, mfccg, mfccs)

In [465]:
MCsimulation

array([0.38248784, 0.38299745, 0.40583739, 0.38299745, 0.38299745,
       0.38299745, 0.38299745, 0.38299745, 0.38299745, 0.38299745,
       0.38299745, 0.38299745, 0.38299745, 0.38299745, 0.38299745,
       0.38304378, 0.38299745, 0.3943016 , 0.38299745, 0.38234885,
       0.3946259 , 0.38299745, 0.38299745, 0.38304378, 0.38299745,
       0.38299745, 0.38299745, 0.38304378, 0.38299745, 0.38299745,
       0.38299745, 0.38299745, 0.38299745, 0.38299745, 0.34681492,
       0.38299745, 0.38299745, 0.38299745, 0.38299745, 0.38299745,
       0.38299745, 0.38299745, 0.38299745, 0.38299745, 0.38299745,
       0.38299745, 0.38299745, 0.38299745, 0.44063007, 0.38299745])

In [466]:
print('Average Hamming Loss')
round(MCsimulation.mean(),4)

Average Hamming Loss


0.3843

In [467]:
print('Standard Error of Hamming Loss')
round(MCsimulation.std(),4)

Standard Error of Hamming Loss


0.0103